# Coursera Peer-graded Assignment
## Segment and Cluster Hillsborough Cty., FL zip codes

### Step 1: Get zip codes from the Capitol Impact Govt. Gateway website
### do any necessary data cleaning and put data into a pandas dataframe

Import libraries to be used 


In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup   # using BeautifulSoup to parse html for data needed for this lab
import json # library to handle json files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from urllib.request import urlopen, Request   # library to read url web page
from urllib.parse import urlencode
import geocoder      #. library to get geographic coordinates of locations
import matplotlib.cm as cm
import matplotlib.colors as colors

#  Import libraries for clustering and map visualization
import folium   # map rendering library
from sklearn.cluster import KMeans   #  library to cluster via k-means
import requests # library to handle requests

Read the wikipedia web page that contains a table of Toronto posstal codes
Parse the page with BeautifulSoup

In [2]:
#
#. Replaced the calls to the web site with the reading of an Excel workbook with the data in it
#. This was done to avoid exceeding call limits to the web site and make it faster to test code
#. The excel workbook was generated directly from the table produced by the web site.
#
#url = 'http://ciclt.net/sn/clt/capitolimpact/gw_ziplist.aspx?ClientCode=capitolimpact&State=fl&StName=Florida&StFIPS=12&FIPS=12057' #URL of the wikipedia page to read
#f = urlopen(url)   # open the url
#wiki_html = f.read()  # read the data from the page
#soup = BeautifulSoup(wiki_html, 'html.parser')  # parse data with BeautifulSoup
hczip_df = pd.read_excel('HC_zipcodes.xlsx', sheet_name = 'zip' )
hczip_df.head()

,ZIP Code,Classification,City,Population
0,33503,P.O. Box,Balm,108
1,33508,P.O. Box,Brandon,0
2,33509,P.O. Box,Brandon,0
3,33510,General,Brandon,27610
4,33511,General,Brandon,52003


#soup # see the data returned from the call baove

Extract the table of postal codes and put into a pandas data frame

table = soup.find_all('table')[0]   #  Extract just the postal code table from the html page
dfs = pd.read_html(str(table))    #pandas gets a list of dataframes for the postal codes

#. Put Hillsborough county zip codes into a pandas dataframe
#. This data was listed in 3 columns on the web page and is returned as 3 separate segments in the list

df = dfs[2]                                                                #. 1st column of the table in the web page
df = df.append(dfs[3], ignore_index=True)            #. 2nd column of the table in the web page
df = df.append(dfs[4], ignore_index=True)            #. 3rd column of the table in the web page

In [3]:
hczip_df.shape

(96, 4)

Now clean up the data frame 
drop any row that does not have a Borough name assigned.
replace any Neighbourhood that is not assigned with thet the Borough name

In [4]:

# Get rid of the zip codes that are assigned to post office boxes

hczip_df2 = hczip_df[hczip_df.Classification!= 'P.O. Box'].copy()
hczip_df2.reset_index(drop=True,inplace = True)

# replace any not assigned Neighbourhoods with Borough

#df_x['Neighbourhood'].loc[df_x['Neighbourhood'] == 'Not assigned'] = df_x['Borough']

hczip_df2.shape

(60, 4)

Finally, group the data by postal code, combine neighbourhoods within postal codes separating them with commas


post_codes = df_x.groupby(['Postcode','Borough'],as_index=False)['Neighbourhood'].apply(','.join).reset_index()  # group postal codes, combine neighborhoods 
post_codes.rename(columns={0:'Neighbourhood'}, inplace=True)

Display the shape of the final data frame

post_codes.shape    #  display the shape of the resulting data frame


post_codes.head(20)


## Get latitude and longtide of each zip code in Hillsborough county FL.

In [5]:
lat_lng_coords = None
latitude = []    # create a list for the latitude
longitude = []  # create a list for the longitude

for index, row in hczip_df2.iterrows():
#    print(row['Zip Code'])
    while(lat_lng_coords == None):
        g = geocoder.arcgis('{}, Hillsborough, FL'.format(row['ZIP Code']))
        lat_lng_coords = g.latlng
    latitude.append(lat_lng_coords[0]) 
    longitude.append(lat_lng_coords[1])
    lat_lng_coords = None

hczip_df2['Latitude'] = latitude
hczip_df2['Longitude'] = longitude
print('coordinates have been loaded')



coordinates have been loaded


In [6]:
hczip_df2


,ZIP Code,Classification,City,Population,Latitude,Longitude
0,33510,General,Brandon,27610,27.951980,-82.320020
1,33511,General,Brandon,52003,27.912380,-82.319210
2,33527,General,Dover,15203,27.992640,-82.216890
3,33534,General,Gibsonton,12839,27.835040,-82.378110
4,33547,General,Lithia,19813,27.838770,-82.174450
5,33548,General,Lutz,5807,28.135350,-82.478070
6,33549,General,Lutz,16132,28.087500,-82.439040
7,33556,General,Odessa,22015,28.144590,-82.597920
8,33558,General,Lutz,20064,28.135230,-82.542760
9,33559,General,Lutz,15427,28.129043,-82.422766


#  ----------------------DEFINE FUNCTIONS----------------------------------------------------
####  Define functions that will be used in this segment of the code

In [7]:
#
# function that extracts the category of the venue
#

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [8]:
#
#  function to get nearby venues in all the neighborhoods
#

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['ZIP Code', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [9]:
#
#  function to sort the venues in descending order.
#

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

####   END OF FUNCTIONS
# -------------------------------END OF FUNCTION DEF's---------------------------------------

####  get coordinates for Hillsborough County, FL to set map center

In [10]:
address = 'Hillsborough county, FL'
lat_lng_coords = None

while(lat_lng_coords == None):
    g = geocoder.arcgis('{}'.format(address))
    lat_lng_coords = g.latlng
    hc_lat = lat_lng_coords[0] 
    hc_lng = lat_lng_coords[1]
    
print('The geograpical coordinate of Hillsborough County are {}, {}.'.format(hc_lat, hc_lng))

The geograpical coordinate of Hillsborough County are 27.930230962000053, -82.30785533999995.


In [11]:
#
# create map of Toronto using latitude and longitude values obtained above
#

map_hc = folium.Map(location=[hc_lat, hc_lng], zoom_start=10)

#
# add markers to map for neighborhoods/postal codes
#

for lat, lng, zipcode, neighborhood in zip(hczip_df2['Latitude'], hczip_df2['Longitude'], hczip_df2['ZIP Code'], hczip_df2['City']):
    label = '{}, {}'.format(zipcode, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_hc)  
    
map_hc


#### Now utilize Foursquare API to explore the neighborhoods.

#### Define Foursquare Credentials and Version

In [12]:
CLIENT_ID = 'KZTP21MDEVOQIJCJPYIIM2LUL1BYATR4CZGVUHVI1N44NHNU' # your Foursquare ID
CLIENT_SECRET = 'AFXBPNNG1TC0BODY5KBSUWC4LL113MGGS5GSZEQSCHHUFYMH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KZTP21MDEVOQIJCJPYIIM2LUL1BYATR4CZGVUHVI1N44NHNU
CLIENT_SECRET:AFXBPNNG1TC0BODY5KBSUWC4LL113MGGS5GSZEQSCHHUFYMH


#### explore the one neighborhood in our dataframe to make sure everything is set up correctly.

In [13]:
hczip_df2.loc[0,'ZIP Code']

33510

In [14]:
hczip_df2.loc[0, 'ZIP Code']
neighborhood_latitude = hczip_df2.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = hczip_df2.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name =hczip_df2.loc[0, 'ZIP Code'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of 33510 are 27.95198000000005, -82.32001999999994.


#### Now, let's get the top 100 venues that are in that neighborhood within a radius of 5 kilometers.

First create the GET request URL for foursquare. 

In [15]:
limit = 100   #Limit to 100 venues
radius = 5000  # 5km  from center point
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    limit)

url

'https://api.foursquare.com/v2/venues/explore?&client_id=KZTP21MDEVOQIJCJPYIIM2LUL1BYATR4CZGVUHVI1N44NHNU&client_secret=AFXBPNNG1TC0BODY5KBSUWC4LL113MGGS5GSZEQSCHHUFYMH&v=20180605&ll=27.95198000000005,-82.32001999999994&radius=5000&limit=100'

#### Get results from foursquare,  in for form of a json file

In [16]:
results_json = requests.get(url).json()
results_json

{'meta': {'code': 200, 'requestId': '5d6fd219396de0002c8f5a8f'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Riverview',
  'headerFullLocation': 'Riverview',
  'headerLocationGranularity': 'city',
  'totalResults': 226,
  'suggestedBounds': {'ne': {'lat': 27.996980045000093,
    'lng': -82.26917209701519},
   'sw': {'lat': 27.906979955000004, 'lng': -82.3708679029847}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bcdfbeafb84c9b612f9223e',
       'name': 'Ulta Beauty',
       'location': {'address': '2442 West Brandon Boulevard',
        'crossStreet': 'Regency Square',
        'lat': 27.941688771699184,
        'lng': -82.32206713492

####   Now that that worked,   put into a pandas dataframe and get just the columns of interest

In [17]:
venues = results_json['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(100)

,name,categories,lat,lng
0,Ulta Beauty,Cosmetics Shop,27.941689,-82.322067
1,Ice Sports Forum,Skating Rink,27.948889,-82.330073
2,Anthony's Coal Fired Pizza,Pizza Place,27.938068,-82.315844
3,Kobe Japanese Steakhouse - Brandon,Japanese Restaurant,27.939480,-82.321710
4,Wawa,Convenience Store,27.938434,-82.310696
5,Barnes & Noble,Bookstore,27.939262,-82.324779
6,Craft Beer Cellar Brandon,Beer Store,27.937067,-82.315972
7,PDQ,Fried Chicken Joint,27.936530,-82.319334
8,Ross Dress for Less,Clothing Store,27.937923,-82.323671
9,Ploy Thai,Thai Restaurant,27.937183,-82.316008


In [18]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


#### Now that that the initial setup works,  let's get venues for all postal codes/neighborhoods

In [19]:
hc_venues = getNearbyVenues (names=hczip_df2['ZIP Code'],
    latitudes=hczip_df2['Latitude'],
    longitudes=hczip_df2['Longitude'],
    radius=5000
     )

33510
33511
33527
33534
33547
33548
33549
33556
33558
33559
33563
33565
33566
33567
33569
33570
33572
33573
33578
33579
33584
33592
33594
33596
33598
33602
33603
33604
33605
33606
33607
33609
33610
33611
33612
33613
33614
33615
33616
33617
33618
33619
33620
33621
33624
33625
33626
33629
33633
33634
33635
33637
33647
33650
33655
33660
33661
33662
33663
33664


In [20]:
hc_venues.head(10)

,ZIP Code,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,33510,27.95198,-82.32002,Ulta Beauty,27.941689,-82.322067,Cosmetics Shop
1,33510,27.95198,-82.32002,Ice Sports Forum,27.948889,-82.330073,Skating Rink
2,33510,27.95198,-82.32002,Anthony's Coal Fired Pizza,27.938068,-82.315844,Pizza Place
3,33510,27.95198,-82.32002,Kobe Japanese Steakhouse - Brandon,27.939480,-82.321710,Japanese Restaurant
4,33510,27.95198,-82.32002,Wawa,27.938434,-82.310696,Convenience Store
5,33510,27.95198,-82.32002,Barnes & Noble,27.939262,-82.324779,Bookstore
6,33510,27.95198,-82.32002,Craft Beer Cellar Brandon,27.937067,-82.315972,Beer Store
7,33510,27.95198,-82.32002,PDQ,27.936530,-82.319334,Fried Chicken Joint
8,33510,27.95198,-82.32002,Ross Dress for Less,27.937923,-82.323671,Clothing Store
9,33510,27.95198,-82.32002,Ploy Thai,27.937183,-82.316008,Thai Restaurant


In [21]:
# How many venues were returned by neighborhoor

hc_venues.groupby('ZIP Code').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
ZIP Code,,,,,,
33510,100,100,100,100,100,100
33511,100,100,100,100,100,100
33527,26,26,26,26,26,26
33534,32,32,32,32,32,32
33547,36,36,36,36,36,36
33548,79,79,79,79,79,79
33549,100,100,100,100,100,100
33556,9,9,9,9,9,9
33558,47,47,47,47,47,47


In [22]:
# How many unique catagories were returned

print('There are {} uniques categories.'.format(len(hc_venues['Venue Category'].unique())))

There are 272 uniques categories.


## Analyze Each Neighborhood

#### encode the venue types to set up for analysis

In [23]:
# one hot encoding
hc_onehot = pd.get_dummies(hc_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
hc_onehot['ZIP Code'] = hc_venues['ZIP Code'] 

# move neighborhood column to the first column
fixed_columns = [hc_onehot.columns[-1]] + list(hc_onehot.columns[:-1])
hc_onehot = hc_onehot[fixed_columns]

hc_onehot.head(10)

,ZIP Code,Adult Boutique,Airport,Airport Lounge,Airport Terminal,American Restaurant,Animal Shelter,Aquarium,Arcade,Arepa Restaurant,...,Vietnamese Restaurant,Warehouse Store,Weight Loss Center,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,33510,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,33510,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,33510,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,33510,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,33510,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,33510,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,33510,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,33510,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,33510,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,33510,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
#. Check what the encoded dataframe looks like.

hc_onehot.shape

(5049, 273)

In [25]:
#. group all venues by postal code

hc_grouped = hc_onehot.groupby('ZIP Code').sum().reset_index()
hc_grouped

,ZIP Code,Adult Boutique,Airport,Airport Lounge,Airport Terminal,American Restaurant,Animal Shelter,Aquarium,Arcade,Arepa Restaurant,...,Vietnamese Restaurant,Warehouse Store,Weight Loss Center,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,33510,0,0,0,0,3,0,0,0,0,...,1,1,0,0,1,0,0,0,0,0
1,33511,0,0,0,0,2,0,0,0,0,...,0,1,0,0,1,1,0,0,0,0
2,33527,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,33534,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,33547,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
5,33548,0,0,0,0,5,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
6,33549,1,0,0,0,2,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
7,33556,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,33558,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,33559,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [26]:
hc_grouped.shape

(60, 273)

In [27]:
#   print top 5 venues for each postal code 

num_top_venues = 5

for hood in hc_grouped['ZIP Code']:
    print("----"+str(hood)+"----")
    temp = hc_grouped[hc_grouped['ZIP Code'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----33510----
                    venue  freq
0             Pizza Place   4.0
1  Furniture / Home Store   4.0
2      Mexican Restaurant   3.0
3     American Restaurant   3.0
4            Burger Joint   3.0


----33511----
                 venue  freq
0       Clothing Store   4.0
1         Burger Joint   4.0
2     Department Store   4.0
3  Japanese Restaurant   4.0
4          Pizza Place   3.0


----33527----
               venue  freq
0        Gas Station   4.0
1       Soccer Field   2.0
2  Convenience Store   2.0
3     Sandwich Place   2.0
4     Farmers Market   2.0


----33534----
            venue  freq
0     Gas Station   4.0
1      Donut Shop   4.0
2  Baseball Field   3.0
3             Bar   2.0
4  Discount Store   2.0


----33547----
                  venue  freq
0  Fast Food Restaurant   4.0
1                  Park   3.0
2     Convenience Store   2.0
3           Wings Joint   2.0
4              Pharmacy   2.0


----33548----
                  venue  freq
0           Pizza Place 

              venue  freq
0              Park   4.0
1    Sandwich Place   3.0
2      Liquor Store   2.0
3               Gym   2.0
4  Airport Terminal   2.0


----33624----
            venue  freq
0   Grocery Store   8.0
1      Restaurant   7.0
2     Pizza Place   5.0
3     Golf Course   4.0
4  Ice Cream Shop   3.0


----33625----
                venue  freq
0  Italian Restaurant   4.0
1       Grocery Store   4.0
2    Sushi Restaurant   4.0
3         Pizza Place   4.0
4            Pharmacy   3.0


----33626----
                  venue  freq
0                   Gym   5.0
1      Sushi Restaurant   5.0
2         Grocery Store   4.0
3   American Restaurant   4.0
4  Fast Food Restaurant   3.0


----33629----
                venue  freq
0       Grocery Store   6.0
1  Mexican Restaurant   6.0
2  Italian Restaurant   5.0
3      Scenic Lookout   4.0
4              Bakery   3.0


----33633----
                 venue  freq
0          Coffee Shop  10.0
1  American Restaurant   7.0
2          Pizza 

#### Let's put that into a *pandas* dataframe

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [28]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['ZIP Code']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['ZIP Code'] = hc_grouped['ZIP Code']

for ind in np.arange(hc_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(hc_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,ZIP Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,33510,Pizza Place,Furniture / Home Store,Fast Food Restaurant,Burger Joint,Cosmetics Shop,American Restaurant,Clothing Store,Bakery,Mexican Restaurant,Coffee Shop
1,33511,Burger Joint,Department Store,Clothing Store,Japanese Restaurant,Grocery Store,Steakhouse,Convenience Store,Cosmetics Shop,Pizza Place,Thai Restaurant
2,33527,Gas Station,Convenience Store,Sandwich Place,Soccer Field,Farmers Market,Intersection,Fast Food Restaurant,Farm,Beer Garden,General Entertainment
3,33534,Gas Station,Donut Shop,Baseball Field,Convenience Store,Bar,Discount Store,Video Store,Fried Chicken Joint,Beer Garden,Light Rail Station
4,33547,Fast Food Restaurant,Park,Video Store,Convenience Store,Wings Joint,Pharmacy,Seafood Restaurant,Sports Club,Boxing Gym,Mobile Phone Shop


## 4. Cluster Neighborhoods

create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [29]:
# set number of clusters
kclusters = 5

hc_grouped_clustering = hc_grouped.drop('ZIP Code', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(hc_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 4, 4, 4, 3, 3, 4, 4, 4], dtype=int32)

In [30]:
hc_grouped_clustering.head()

,Adult Boutique,Airport,Airport Lounge,Airport Terminal,American Restaurant,Animal Shelter,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,...,Vietnamese Restaurant,Warehouse Store,Weight Loss Center,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,0,0,0,0,3,0,0,0,0,0,...,1,1,0,0,1,0,0,0,0,0
1,0,0,0,0,2,0,0,0,0,0,...,0,1,0,0,1,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0


In [31]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
hc_merged = hczip_df2
hc_merged = hc_merged.join(neighborhoods_venues_sorted.set_index('ZIP Code'), on='ZIP Code')

In [32]:
hc_merged.head()

,ZIP Code,Classification,City,Population,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,33510,General,Brandon,27610,27.95198,-82.32002,Pizza Place,Furniture / Home Store,Fast Food Restaurant,Burger Joint,Cosmetics Shop,American Restaurant,Clothing Store,Bakery,Mexican Restaurant,Coffee Shop
1,33511,General,Brandon,52003,27.91238,-82.31921,Burger Joint,Department Store,Clothing Store,Japanese Restaurant,Grocery Store,Steakhouse,Convenience Store,Cosmetics Shop,Pizza Place,Thai Restaurant
2,33527,General,Dover,15203,27.99264,-82.21689,Gas Station,Convenience Store,Sandwich Place,Soccer Field,Farmers Market,Intersection,Fast Food Restaurant,Farm,Beer Garden,General Entertainment
3,33534,General,Gibsonton,12839,27.83504,-82.37811,Gas Station,Donut Shop,Baseball Field,Convenience Store,Bar,Discount Store,Video Store,Fried Chicken Joint,Beer Garden,Light Rail Station
4,33547,General,Lithia,19813,27.83877,-82.17445,Fast Food Restaurant,Park,Video Store,Convenience Store,Wings Joint,Pharmacy,Seafood Restaurant,Sports Club,Boxing Gym,Mobile Phone Shop


In [33]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

hc_merged = hczip_df2

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
hc_merged = hc_merged.join(neighborhoods_venues_sorted.set_index('ZIP Code'), on='ZIP Code')

# Some neighborhoods did not return any venues,  drop those neighborhoods from the clustering
# Because some had NaN for cluster labels, column was float and needs to be integer so convert to int after dropping NaN rows

#toronto_merged.dropna(subset=['Cluster Labels'],inplace=True)
#toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype('Int32')
#toronto_merged['Cluster Labels'].dtype




In [34]:
hc_merged.head(10)

,ZIP Code,Classification,City,Population,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,33510,General,Brandon,27610,27.951980,-82.320020,3,Pizza Place,Furniture / Home Store,Fast Food Restaurant,Burger Joint,Cosmetics Shop,American Restaurant,Clothing Store,Bakery,Mexican Restaurant,Coffee Shop
1,33511,General,Brandon,52003,27.912380,-82.319210,3,Burger Joint,Department Store,Clothing Store,Japanese Restaurant,Grocery Store,Steakhouse,Convenience Store,Cosmetics Shop,Pizza Place,Thai Restaurant
2,33527,General,Dover,15203,27.992640,-82.216890,4,Gas Station,Convenience Store,Sandwich Place,Soccer Field,Farmers Market,Intersection,Fast Food Restaurant,Farm,Beer Garden,General Entertainment
3,33534,General,Gibsonton,12839,27.835040,-82.378110,4,Gas Station,Donut Shop,Baseball Field,Convenience Store,Bar,Discount Store,Video Store,Fried Chicken Joint,Beer Garden,Light Rail Station
4,33547,General,Lithia,19813,27.838770,-82.174450,4,Fast Food Restaurant,Park,Video Store,Convenience Store,Wings Joint,Pharmacy,Seafood Restaurant,Sports Club,Boxing Gym,Mobile Phone Shop
5,33548,General,Lutz,5807,28.135350,-82.478070,3,Pizza Place,American Restaurant,Grocery Store,Sandwich Place,Gym / Fitness Center,Donut Shop,Chinese Restaurant,Pharmacy,Bank,Smoothie Shop
6,33549,General,Lutz,16132,28.087500,-82.439040,3,Sandwich Place,Grocery Store,Bar,College Cafeteria,Bubble Tea Shop,Convenience Store,Sushi Restaurant,Fast Food Restaurant,Taco Place,Donut Shop
7,33556,General,Odessa,22015,28.144590,-82.597920,4,Convenience Store,Sandwich Place,Farm,Donut Shop,Trail,Gym,Gym / Fitness Center,Grocery Store,American Restaurant,Dessert Shop
8,33558,General,Lutz,20064,28.135230,-82.542760,4,Golf Course,Sandwich Place,Furniture / Home Store,Gym / Fitness Center,Pizza Place,Grocery Store,Gas Station,Church,Beer Garden,Supplement Shop
9,33559,General,Lutz,15427,28.129043,-82.422766,4,Gym,Grocery Store,Sandwich Place,Pet Store,Discount Store,Italian Restaurant,Bank,Chinese Restaurant,Salon / Barbershop,Asian Restaurant


In [35]:
# create map
map_clusters = folium.Map(location=[hc_lat, hc_lng], zoom_start=11)
# toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].dtype=pd.Int64Dtype()


# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(hc_merged['Latitude'], hc_merged['Longitude'], hc_merged['ZIP Code'], hc_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [36]:
hc_merged.loc[hc_merged['Cluster Labels'] == 0, hc_merged.columns[[0] + list(range(5, hc_merged.shape[1]))]]

,ZIP Code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,33563,-82.128920,0,Fast Food Restaurant,Gas Station,Discount Store,Sandwich Place,Pizza Place,BBQ Joint,Grocery Store,Donut Shop,Mexican Restaurant,Gym / Fitness Center
11,33565,-82.121480,0,Fast Food Restaurant,BBQ Joint,Gas Station,Intersection,Sandwich Place,Discount Store,Pizza Place,Park,Italian Restaurant,Pharmacy
12,33566,-82.140700,0,Fast Food Restaurant,Discount Store,Pizza Place,Gas Station,Sandwich Place,Mexican Restaurant,Grocery Store,BBQ Joint,Convenience Store,Donut Shop
17,33573,-82.362070,0,Grocery Store,Gas Station,Fast Food Restaurant,Discount Store,Golf Course,American Restaurant,Sandwich Place,Food,Fried Chicken Joint,Big Box Store
20,33584,-82.301090,0,Gas Station,Pizza Place,Fast Food Restaurant,Discount Store,Hotel,Pharmacy,Sandwich Place,Grocery Store,Video Store,Convenience Store
22,33594,-82.227430,0,Pizza Place,Grocery Store,Pharmacy,Fast Food Restaurant,American Restaurant,Sandwich Place,Mobile Phone Shop,Salon / Barbershop,Indian Restaurant,Tex-Mex Restaurant
37,33615,-82.581720,0,Fast Food Restaurant,Pizza Place,Coffee Shop,Trail,Gas Station,Bar,Grocery Store,Pharmacy,Diner,Chinese Restaurant
38,33616,-82.525890,0,Grocery Store,Pizza Place,Pharmacy,Sandwich Place,Gym,Nail Salon,Restaurant,Gas Station,Scenic Lookout,American Restaurant
41,33619,-82.380415,0,Convenience Store,Discount Store,Gas Station,Sandwich Place,Pizza Place,Video Store,Grocery Store,Donut Shop,Chinese Restaurant,Pharmacy


Cluster 1 = midtown tampa area

In [37]:
hc_merged.loc[hc_merged['Cluster Labels'] == 1, hc_merged.columns[[0] + list(range(5, hc_merged.shape[1]))]]

,ZIP Code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,33602,-82.45364,1,Coffee Shop,American Restaurant,Pizza Place,Theater,Park,Italian Restaurant,Brewery,Bar,Smoke Shop,Restaurant
26,33603,-82.45728,1,Brewery,Pizza Place,American Restaurant,Coffee Shop,Smoke Shop,Bar,Ice Cream Shop,Sushi Restaurant,Latin American Restaurant,Park
28,33605,-82.43830,1,Brewery,Pizza Place,Coffee Shop,Smoke Shop,American Restaurant,Bar,Park,Nightclub,Italian Restaurant,New American Restaurant
29,33606,-82.47628,1,Coffee Shop,Mexican Restaurant,Restaurant,American Restaurant,Park,Pizza Place,Scenic Lookout,Sandwich Place,Theater,Italian Restaurant
32,33610,-82.42249,1,Brewery,Bar,Pizza Place,Coffee Shop,American Restaurant,Smoke Shop,Ice Cream Shop,Plaza,BBQ Joint,Restaurant
48,33633,-82.45927,1,Coffee Shop,American Restaurant,Park,Pizza Place,Theater,Bar,Restaurant,Hockey Arena,Scenic Lookout,Sandwich Place
53,33650,-82.45927,1,Coffee Shop,American Restaurant,Park,Pizza Place,Theater,Bar,Restaurant,Hockey Arena,Scenic Lookout,Sandwich Place
54,33655,-82.45927,1,Coffee Shop,American Restaurant,Park,Pizza Place,Theater,Bar,Restaurant,Hockey Arena,Scenic Lookout,Sandwich Place
55,33660,-82.45927,1,Coffee Shop,American Restaurant,Park,Pizza Place,Theater,Bar,Restaurant,Hockey Arena,Scenic Lookout,Sandwich Place
56,33661,-82.45927,1,Coffee Shop,American Restaurant,Park,Pizza Place,Theater,Bar,Restaurant,Hockey Arena,Scenic Lookout,Sandwich Place


Cluster 2 = Theme park area around Busch Gardens/Adventure Island

In [38]:
hc_merged.loc[hc_merged['Cluster Labels'] == 2, hc_merged.columns[[0] + list(range(5, hc_merged.shape[1]))]]

,ZIP Code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,33604,-82.44351,2,Theme Park,Zoo,Theme Park Ride / Attraction,Bar,Ice Cream Shop,Zoo Exhibit,Convenience Store,Diner,Restaurant,Fast Food Restaurant
34,33612,-82.43868,2,Theme Park Ride / Attraction,Zoo,Theme Park,Coffee Shop,Bar,Sandwich Place,Convenience Store,Bubble Tea Shop,Deli / Bodega,Playground
35,33613,-82.41690,2,Coffee Shop,Sandwich Place,Fast Food Restaurant,College Cafeteria,Theme Park,Bubble Tea Shop,Grocery Store,American Restaurant,Mexican Restaurant,Restaurant
39,33617,-82.40051,2,Theme Park,Theme Park Ride / Attraction,Zoo,Hookah Bar,Grocery Store,Mexican Restaurant,Fast Food Restaurant,Restaurant,Park,Convenience Store
42,33620,-82.41003,2,Theme Park,Theme Park Ride / Attraction,Coffee Shop,Zoo,Fast Food Restaurant,Sandwich Place,College Cafeteria,Bagel Shop,Italian Restaurant,Restaurant
51,33637,-82.37233,2,Theme Park,Convenience Store,Hookah Bar,Coffee Shop,Mexican Restaurant,Fast Food Restaurant,Zoo,Park,Restaurant,Grocery Store


Cluster 3 is older suburbs, west side of city primarily.

In [39]:
hc_merged.loc[hc_merged['Cluster Labels'] == 3, hc_merged.columns[[0] + list(range(5, hc_merged.shape[1]))]]

,ZIP Code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,33510,-82.32002,3,Pizza Place,Furniture / Home Store,Fast Food Restaurant,Burger Joint,Cosmetics Shop,American Restaurant,Clothing Store,Bakery,Mexican Restaurant,Coffee Shop
1,33511,-82.31921,3,Burger Joint,Department Store,Clothing Store,Japanese Restaurant,Grocery Store,Steakhouse,Convenience Store,Cosmetics Shop,Pizza Place,Thai Restaurant
5,33548,-82.47807,3,Pizza Place,American Restaurant,Grocery Store,Sandwich Place,Gym / Fitness Center,Donut Shop,Chinese Restaurant,Pharmacy,Bank,Smoothie Shop
6,33549,-82.43904,3,Sandwich Place,Grocery Store,Bar,College Cafeteria,Bubble Tea Shop,Convenience Store,Sushi Restaurant,Fast Food Restaurant,Taco Place,Donut Shop
18,33578,-82.34449,3,American Restaurant,Grocery Store,Fast Food Restaurant,Burger Joint,Coffee Shop,Bar,Hardware Store,Breakfast Spot,Gym / Fitness Center,Pizza Place
23,33596,-82.26273,3,Pizza Place,Pharmacy,American Restaurant,Gym,Mexican Restaurant,Sushi Restaurant,Grocery Store,Ice Cream Shop,Sandwich Place,Mobile Phone Shop
30,33607,-82.51787,3,American Restaurant,Seafood Restaurant,Italian Restaurant,Steakhouse,Hotel,Cosmetics Shop,Mexican Restaurant,Fast Food Restaurant,Grocery Store,Lingerie Store
31,33609,-82.49129,3,Mexican Restaurant,Coffee Shop,American Restaurant,Italian Restaurant,Grocery Store,Restaurant,Sandwich Place,Scenic Lookout,BBQ Joint,Park
33,33611,-82.51317,3,Grocery Store,Scenic Lookout,Restaurant,Italian Restaurant,Bar,Pizza Place,Gym / Fitness Center,Spa,Ice Cream Shop,Breakfast Spot
36,33614,-82.50210,3,American Restaurant,Latin American Restaurant,Zoo Exhibit,Asian Restaurant,Diner,Pharmacy,Coffee Shop,South American Restaurant,Spanish Restaurant,Bar


In [41]:
Cluster 4 = outer suburbs, newer, fast growing.

SyntaxError: invalid syntax (<ipython-input-41-01a7049d1d27>, line 1)

In [40]:
hc_merged.loc[hc_merged['Cluster Labels'] == 4, hc_merged.columns[[0] + list(range(5, hc_merged.shape[1]))]]

,ZIP Code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,33527,-82.216890,4,Gas Station,Convenience Store,Sandwich Place,Soccer Field,Farmers Market,Intersection,Fast Food Restaurant,Farm,Beer Garden,General Entertainment
3,33534,-82.378110,4,Gas Station,Donut Shop,Baseball Field,Convenience Store,Bar,Discount Store,Video Store,Fried Chicken Joint,Beer Garden,Light Rail Station
4,33547,-82.174450,4,Fast Food Restaurant,Park,Video Store,Convenience Store,Wings Joint,Pharmacy,Seafood Restaurant,Sports Club,Boxing Gym,Mobile Phone Shop
7,33556,-82.597920,4,Convenience Store,Sandwich Place,Farm,Donut Shop,Trail,Gym,Gym / Fitness Center,Grocery Store,American Restaurant,Dessert Shop
8,33558,-82.542760,4,Golf Course,Sandwich Place,Furniture / Home Store,Gym / Fitness Center,Pizza Place,Grocery Store,Gas Station,Church,Beer Garden,Supplement Shop
9,33559,-82.422766,4,Gym,Grocery Store,Sandwich Place,Pet Store,Discount Store,Italian Restaurant,Bank,Chinese Restaurant,Salon / Barbershop,Asian Restaurant
13,33567,-82.163790,4,Clothing Store,Intersection,Gas Station,Gym,Golf Course,Soccer Field,Pet Store,Park,Pizza Place,Convenience Store
14,33569,-82.320110,4,Gas Station,Convenience Store,Donut Shop,Fast Food Restaurant,American Restaurant,Video Store,Grocery Store,Bar,Big Box Store,Café
15,33570,-82.432960,4,American Restaurant,Fast Food Restaurant,Grocery Store,Seafood Restaurant,Sandwich Place,Discount Store,Breakfast Spot,Rest Area,Food & Drink Shop,Mexican Restaurant
16,33572,-82.400000,4,Grocery Store,Pizza Place,American Restaurant,Construction & Landscaping,Donut Shop,Restaurant,Fast Food Restaurant,Scenic Lookout,Gym / Fitness Center,Spa
